# Greedy coloring and the coloring number

### The Coloring Number of a Graph

The **coloring number** of a graph $G$, denoted as $Col(G)$, is a parameter that provides an **upper bound** on the minimum number of colors required to properly color the graph. It is defined based on a **well-ordering** of the vertices of G.

---

It is enough to apply such an order of deletion of vertices $v_n, v_{n-1}, \ldots, v_1$, that vertex $v_i$ is the vertex with the smallest degree in $$G[V(G) \setminus \{v_n, v_{n-1}, \ldots, v_{i+1}\}]$$ Then $v_1, v_2, \ldots, v_n$ is a good colouring order.

In [3]:
import pygame
import time

# We set the exemplary graph as an adjacency list
graph = {
    0: [1, 2],
    1: [0, 2, 3],
    2: [0, 1, 3],
    3: [1, 2, 4],
    4: [3]
}

# Positions of nodes for visualization (pygame)
positions = {
    0: (100, 100),
    1: (200, 50),
    2: (200, 150),
    3: (300, 100),
    4: (400, 100)
}

def greedy_coloring(graph, vertex_order):
    node_colors = {}
    steps = []

    for node in vertex_order:
        neighbor_colors = {node_colors[neighbor] for neighbor in graph[node] if neighbor in node_colors}
        color = 0
        while color in neighbor_colors:
            color += 1
        node_colors[node] = color

        steps.append((node, color, neighbor_colors))

    return node_colors, steps


# Choose the best vertex ordering using the smallest last vertex ordering (vertices with the smallest degrees go last)
def smallest_last_ordering(graph):
    ordering = []
    remaining = set(graph.keys())

    while remaining:
        # Find the vertex with the smallest degree
        min_node = min(remaining, key=lambda node: len([n for n in graph[node] if n in remaining]))
        ordering.append(min_node)
        remaining.remove(min_node)

    return ordering[::-1]  # Reverse the order to get the smallest last ordering


# Pygame setup for visualization
def visualize_coloring(graph, positions, steps, vertex_order):
    pygame.init()
    screen = pygame.display.set_mode((600, 400))
    pygame.display.set_caption("Interactive Greedy Graph Coloring with Vertex Ordering")

    palette = [
        (255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0),
        (255, 0, 255), (0, 255, 255), (128, 128, 128), (255, 165, 0)
    ]
    white = (255, 255, 255)
    black = (0, 0, 0)
    gray = (200, 200, 200)

    font = pygame.font.Font(None, 24)

    def draw_graph(node_colors, current_node=None, neighbor_colors=None):
        screen.fill(white)

        for node, neighbors in graph.items():
            for neighbor in neighbors:
                pygame.draw.line(screen, black, positions[node], positions[neighbor], 1)

        for node, (x, y) in positions.items():
            color = palette[node_colors.get(node, -1)] if node in node_colors else gray
            pygame.draw.circle(screen, color, (x, y), 20)
            pygame.draw.circle(screen, black, (x, y), 20, 1)
            label = font.render(str(node), True, black)
            screen.blit(label, (x - 10, y - 10))

            if node == current_node:
                pygame.draw.circle(screen, (0, 255, 255), (x, y), 25, 2)

            if neighbor_colors and node in graph[current_node]:
                neighbor_color_text = font.render(f"Color: {node_colors.get(node, 'None')}", True, black)
                screen.blit(neighbor_color_text, (x - 30, y + 30))

        pygame.display.flip()

    # Pygame animation loop
    node_colors = {}
    running = True
    for i, (node, color, neighbor_colors) in enumerate(steps):
        if not running:
            break

        draw_graph(node_colors, current_node=node, neighbor_colors=neighbor_colors)

        available_colors = font.render(f"Step {i + 1}: Node {node}, Color {color}", True, black)
        sorted_sequence = font.render(f"Vertex Order: {vertex_order}", True, black)
        screen.blit(available_colors, (20, 350))
        screen.blit(sorted_sequence, (20, 320))
        pygame.display.flip()

        waiting = True
        while waiting:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False
                    waiting = False
                if event.type == pygame.KEYDOWN:
                    waiting = False

        node_colors[node] = color
        draw_graph(node_colors)

    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

    pygame.quit()

vertex_order = smallest_last_ordering(graph)
colors, steps = greedy_coloring(graph, vertex_order)
visualize_coloring(graph, positions, steps, vertex_order)

The smallest_last_ordering function finds the vertex with the smallest degree at each step. For each node, it calculates the degree of all remaining nodes. The time complexity for this step is $O(V^2)$, where $V$ is the number of vertices, because in the worst case, finding the minimum degree node involves scanning all remaining nodes, and for each of those, checking the neighbors. 

---

For greedy colouring, the time compelxity is $O(V + E)$, where $E$ is the number of edges in the graph (since for each node, we potentially look at all of its neighbors).